<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

#  Data Connections Exercise

## Ask a Legal Research Assistant Bot about the US Constitution

Your function should do the following:

* Read the US_Constitution.txt file inside the some_data folder
* Split this into chunks (you choose the size)
* Write this to a ChromaDB Vector Store
* Use Context Compression to return the relevant portion of the document to the question

In [2]:
# Build a sample vectorDB
from langchain.vectorstores import Chroma
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor 


In [3]:
import os
embedding_function = OpenAIEmbeddings()

In [13]:
def us_constitution_helper(question):
    '''
    Takes in a question about the US Constitution and returns the most relevant
    part of the constitution. Notice it may not directly answer the actual question!
    
    Follow the steps below to fill out this function:
    '''
    # PART ONE:
    # LOAD "some_data/US_Constitution in a Document object
    loader = TextLoader("/home/ankdesh/explore/LearnTry-ML/llm/Udemy-LangChain/01-Data-Connections/some_data/US_Constitution.txt") 
    doc = loader.load()
    
    # PART TWO
    # Split the document into chunks (you choose how and what size)
    # split it into chunks
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(chunk_size=100, )
    docs = text_splitter.split_documents(doc)

    # PART THREE
    # EMBED THE Documents (now in chunks) to a persisted ChromaDB
    db = Chroma.from_documents(docs, embedding_function) 

    # PART FOUR
    # Use ChatOpenAI and ContextualCompressionRetriever to return the most
    # relevant part of the documents.
    model = ChatOpenAI(api_key=os.getenv('OPENAI_API_KEY'), temperature=0)
    compressor = LLMChainExtractor.from_llm(model)
    compression_retriever = ContextualCompressionRetriever(base_compressor=compressor, base_retriever=db.as_retriever()) 
    compressed_docs = compression_retriever.get_relevant_documents(question)
    ret_val = "\n".join(x.page_content for x in compressed_docs)
    return ret_val
    

## Example Usage:

Notice how it doesn't return an entire Document of a large chunk size, but instead the "compressed" version!

In [14]:
print(us_constitution_helper("What is the 13th Amendment?"))

ValueError: Got a larger chunk overlap (200) than chunk size (100), should be smaller.